<a href="https://colab.research.google.com/github/VishanOberoi/Torch/blob/main/RNN_GRU_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data  import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

#Create the RNN

In [26]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first= True)
    self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    out, _ =  self.rnn(x, h0)
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)
    return out


#Creating a GRU

In [45]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first= True)
    self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    out, _ =  self.gru(x, h0)
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)
    return out


#Create an LSTM

In [67]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first= True)
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    out, _ =  self.lstm(x, (h0, c0))
    out = self.fc(out[:, -1, :] )
    return out


# Hyperparams

In [68]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 28
sequence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 5
device

device(type='cuda')

#Load Data

In [69]:
train_dataset = datasets.MNIST(root = 'dataset/', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size , shuffle = True)

test_dataset =  datasets.MNIST(root = 'dataset/', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size , shuffle = True)


# Initialize the network



In [70]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [71]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)


#Train the network


In [72]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device = device).squeeze(1) # Remove axis 1
    targets = targets.to(device = device)


    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    #backward
    optimizer.zero_grad() #set all gradients to zero for each new batch
    loss.backward()

    #Adam step
    optimizer.step() #Update weights









In [73]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking acc on training data")
  else:
    print("Checking acc on testing data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad(): #Dont need to compute the gradients
    for x,y in loader:
      x = x.to(device = device).squeeze(1)
      y = y.to(device = device)

      scores = model(x) #There are 10 digits, give the max prob
      scores.max(1) #Shape is 64x10, We want vals for 10 part
      _, preds = scores.max(1)
      num_correct +=  (preds == y).sum()
      num_samples += preds.size(0)
    #Convert the tensors to floats
    print(f"Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}")

  model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


Checking acc on training data
Got 59292/ 60000 with accuracy 98.82
Checking acc on testing data
Got 9867/ 10000 with accuracy 98.67


# testing acc of 97.54 with a simple RNN
# testing acc of 98.73 with GRU
# testing acc of 98.94 with LSTM